In [74]:
from datetime import datetime
from elasticsearch import Elasticsearch, RequestsHttpConnection
import nbformat as nbf

This notebook is used to enable search across all the notebooks if you don't want to use grep or something simliar

<b>How to set this up</b>

To start an elastic sever
<code>
    docker network create elastic
    docker pull docker.elastic.co/elasticsearch/elasticsearch:7.13.1
    docker run --name es01-test --net elastic -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.13.1
    </code>
    
    
To start Kibana: 
<code>
docker pull docker.elastic.co/kibana/kibana:7.13.1
docker run --name kib01-test --net elastic -p 5601:5601 -e "ELASTICSEARCH_HOSTS=http://es01-test:9200" docker.elastic.co/kibana/kibana:7.13.1
    </code>

In [118]:
import requests
res = requests.get('host.docker.internal:9200')
print(res.content)

InvalidSchema: No connection adapters were found for 'host.docker.internal:9200'

In [16]:
# Connect to ES
es = Elasticsearch(hosts=[{"host": "host.docker.internal", "port": 9200}], 
                   connection_class=RequestsHttpConnection, max_retries=30,
                       retry_on_timeout=True, request_timeout=30)

In [54]:
NB_VERSION = 4

def extract_text(notebook_str):
    formatted = nbf.read(notebook_str, as_version=NB_VERSION)
    text = []
    for cell in formatted.get('cells', []):
        if 'source' in cell and 'cell_type' in cell:
            #print("HEELL")
            if cell['cell_type'] == 'code' or cell['cell_type'] == 'markdown':
                text.append(cell['source'])

    return(text)

In [99]:
doc = extract_text("../work/HTMNotebooks/HTM_Overview_5.ipynb")

In [106]:
elasticDBName = "notebook-search"

listOfStringsInNotebook = []
for i in range(len(doc)):
    cellDict = {}
    cellDict['text'] =  doc[i],
    cellDict['timestamp'] =  datetime.now()
    listOfStringsInNotebook.append(cellDict)
    es.index(index= elasticDBName, body=cellDict)
    


In [116]:

#let's find darth vader
es.search(index=elasticDBName, body={"query": {"prefix" : { "text" : "HTM" }}})

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [107]:
es.indices.refresh(index=elasticDBName)

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [114]:
res = es.search(index=elasticDBName, body={"query": {"match_all": {}}})
res

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 146, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'notebook-search',
    '_type': '_doc',
    '_id': 'i1dGCHoBnLMRWPwbU-cV',
    '_score': 1.0,
    '_source': {'text': ["It quickly get's large. I accupte that 30 out of 40 bits will match my original SDR, than the number that will match will very large.  "],
     'timestamp': '2021-06-14T02:01:57.024856'}},
   {'_index': 'notebook-search',
    '_type': '_doc',
    '_id': 'jFdGCHoBnLMRWPwblOdV',
    '_score': 1.0,
    '_source': {'text': ['# overap - is "spaces"\n# offbits - only 1 way all offbits can can fit \n\ndef compute_overlap_set_cardinality(n, w0, w1, b, provide_summary = True):\n    on_bit_space = sp.binomial(w0, b)\n    off_bit_space = sp.binomial(n - w0, w1 - b)\n    overlap_set = on_bit_space * off_bit_space\n    if provide_summary:\n        print("Given a capcity of", str(w0), "b

In [102]:


#res = es.index(index= elasticDBName, body=l[20])
#print(res['result'])

#res = es.get(index= elasticDBName, id=1)


#res.indices.refresh(index=elasticDBName)

#res = es.search(index=elasticDBName, body={"query": {"match_all": {}}})


# print("Got %d Hits:"aa % res['hits']['total']['value'])
# for hit in res['hits']['hits']:
#     print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

<b>Searching</b>

In [117]:
import requests
res = requests.get('http://localhost:9200')
print(res.content)

ConnectionError: HTTPConnectionPool(host='localhost', port=9200): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f3f9f633160>: Failed to establish a new connection: [Errno 111] Connection refused'))